## 1. 라이브러리 임포트

In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.autograd import Variable
import numpy as np

## 2. 하이퍼파라미터 세팅

In [2]:
batch_size=128
num_epochs=10

word_vec_size=256
dropout_p=0.3

hidden_size=512
num_layers=4

learning_rate=0.001

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 53 kB 2.5 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


## 3. SMS train, test dataset 가져오기

In [5]:
from torchtext import data
from data_loader import DataLoader

In [6]:
loaders = DataLoader(
    train_fn='./sms.maxlen.uniq.shuf.train.tsv',
    batch_size=batch_size,
    valid_ratio=.2,
    device=-1,
    max_vocab=999999,
    min_freq=5)

In [7]:
test_loaders = DataLoader(
    train_fn='./sms.maxlen.uniq.shuf.test.tsv',
    batch_size=batch_size,
    valid_ratio=.01,
    device=-1,
    max_vocab=999999,
    min_freq=5)

## 4. 대략적인 데이터 형태 확인

In [8]:
print("|train| =", len(loaders.train_loader.dataset),
     '|valid| =', len(loaders.valid_loader.dataset))

vocab_size=len(loaders.text.vocab)
num_classes=len(loaders.label.vocab)
print('|vobab| =', vocab_size, '|classes| =', num_classes)

|train| = 3722 |valid| = 930
|vobab| = 1533 |classes| = 2


## 5. 데이터 로드함수

In [9]:
n=3
for i, data in enumerate(loaders.train_loader):
    labels=data.label
    texts=data.text
    
    if i>n:
        break
    
    print ("[%d]" %i)
    print(" 한 번에 로드되는 데이터 크기:", len(labels))
    
    for j in range(n):
        label=labels[j].numpy()
        text=texts[j].numpy()
        print("label: ",label)
        print("text: ", text.shape)

[0]
 한 번에 로드되는 데이터 크기: 128
label:  0
text:  (10,)
label:  0
text:  (10,)
label:  0
text:  (10,)
[1]
 한 번에 로드되는 데이터 크기: 128
label:  0
text:  (4,)
label:  0
text:  (4,)
label:  0
text:  (4,)
[2]
 한 번에 로드되는 데이터 크기: 128
label:  0
text:  (7,)
label:  0
text:  (7,)
label:  0
text:  (7,)
[3]
 한 번에 로드되는 데이터 크기: 10
label:  0
text:  (60,)
label:  0
text:  (60,)
label:  0
text:  (60,)


## 6. 모델 선언

In [10]:
class RNN(nn.Module):
    def __init__(self,
                 input_size,
                 word_vec_size,
                 hidden_size,
                 n_classes,
                 num_layers=4,
                 dropout_p=0.3):
        
        super(RNN, self).__init__()
        
        self.input_size=input_size
        self.word_vec_size=word_vec_size
        self.hidden_size=hidden_size
        self.n_classes=n_classes
        self.num_layers=num_layers
        self.dropout_p = dropout_p
        
        self.emb=nn.Embedding(input_size, word_vec_size)
        
        self.lstm=nn.LSTM(input_size=word_vec_size,
                          hidden_size=hidden_size, 
                          num_layers=num_layers,
                          dropout = dropout_p,
                          batch_first=True,
                          bidirectional=True)
        self.fc=nn.Linear(hidden_size*2, num_classes)
        self.activation=nn.LogSoftmax(dim=-1)
        
    def forward(self,x):
        x=self.emb(x)
        x, _ = self.lstm(x)
        
        out=self.activation(self.fc(x[:, -1]))
        
        return out

In [11]:
model = RNN(input_size=vocab_size,
            word_vec_size=word_vec_size,
            hidden_size=hidden_size,
            n_classes=num_classes,
            num_layers=num_layers,
            dropout_p=dropout_p)

In [12]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    
    model.eval()
    
    for i, data in enumerate(dloader):
        texts=data.text.to(device)
        labels=data.label.to(device)
         
        output=model(texts)
        _, output_index = torch.max(output,1)
        
        total += labels.size(0)
        correct += (output_index == labels).sum().float()
        
    model.train()
    return (100*correct/total).numpy()

In [13]:
print("Accuracy of test data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of test data: 13.01


## 7. loss, optimizer

In [15]:
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## 8. 학습

In [18]:
total_step = len(loaders.train_loader)
for epoch in range(num_epochs):
    for i, data in enumerate(loaders.train_loader):
        texts=data.text.to(device)
        labels=data.label.to(device)
        
        print("[%d]" %i)
        
        outputs=model(texts)
        loss=loss_func(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1)%10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accr: {:.2f}'
                 .format(epoch+1, num_epochs, i+1, total_step, loss.item(),
                        ComputeAccr(loaders.valid_loader, model)))

[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [1/10], Step [10/30], Loss: 1.1330, Accr: 86.99
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [1/10], Step [20/30], Loss: 1.2169, Accr: 86.99
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [1/10], Step [30/30], Loss: 1.3768, Accr: 86.99
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [2/10], Step [10/30], Loss: 0.9219, Accr: 86.88
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [2/10], Step [20/30], Loss: 0.2145, Accr: 86.99
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [2/10], Step [30/30], Loss: 0.6151, Accr: 88.92
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [3/10], Step [10/30], Loss: 0.2677, Accr: 92.04
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
Epoch [3/10], Step [20/30], Loss: 0.0324, Accr: 90.97
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
Epoch [3/10], Step [30/30], Loss: 0.0558, Accr: 93.12
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
Epoch [4/10], Step [10/30], Loss: 0.2058, Accr: 88.92


## 9. 테스트

In [19]:
print("Accuracy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Valid Data: 96.77


## 10. 학습된 파라미터 저장

In [22]:
netname = './rnn_weight.pkl'
torch.save(model, netname, )

## 11. 학습된 파라미터 로드

In [23]:
netname = './rnn_weight.pkl'
model = torch.load(netname)

print("Accuracy of Valid Data: %.2f" %ComputeAccr(loaders.valid_loader, model))

Accuracy of Valid Data: 96.77
